In [1]:
from transformers import AutoTokenizer ,AutoModelForCausalLM
import torch
import re
from sklearn.utils import shuffle

In [2]:
# util function to get expected len after tokenizing
def get_length_param(text: str, tokenizer) -> str:
    tokens_count = len(tokenizer.encode(text))
    if tokens_count <= 15:
        len_param = '1'
    elif tokens_count <= 50:
        len_param = '2'
    elif tokens_count <= 256:
        len_param = '3'
    else:
        len_param = '-'
    return len_param

In [7]:
def remove_duplicates(S):
    S = re.sub(r'[a-zA-Z]+', '', S) #Remove english
    S = S.split()
    result = ""
    for subst in S:
        if subst not in result:
            result += subst+" "
    return result.rstrip()

In [3]:
fit_checkpoint = "WarBot"
tokenizer = AutoTokenizer.from_pretrained(fit_checkpoint)
model = AutoModelForCausalLM.from_pretrained(fit_checkpoint)

In [12]:
quote = "Однажды мы проснёмся и поймём, что бригада Нахаль наваляла десантникам по самые помидоры"

In [72]:
# encode the input, add the eos_token and return a tensor in Pytorch
user_inpit_ids = tokenizer.encode(f"|0|{get_length_param(quote, tokenizer)}|" \
                                              + quote + tokenizer.eos_token, return_tensors="pt")

#chat_history_ids = torch.cat([chat_history_ids, user_inpit_ids], dim=-1)

chat_history_ids = user_inpit_ids # To be changed

output_id = model.generate(
            chat_history_ids,
            num_return_sequences=1, # use for more variants, but have to print [i]
            max_length=300, #512
            no_repeat_ngram_size=1, #3
            do_sample=True, #True
            top_k=50,#50
            top_p=0.9, #0.9
            temperature = 0.45, # was 0.6, 0 for greedy
            #mask_token_id=tokenizer.mask_token_id,
            eos_token_id=tokenizer.eos_token_id,
            #unk_token_id=tokenizer.unk_token_id,
            pad_token_id=tokenizer.pad_token_id,
            #pad_token_id=tokenizer.eos_token_id,
            #device='cpu'
        )

In [59]:
def removeSigns(S):
    last_index = max(S.rfind("."), S.rfind("!"))
    if last_index >= 0:
        S = S[:last_index+1]
    return S

In [63]:
def getResponce():
    response = tokenizer.decode(output_id[0], skip_special_tokens=True)
    response = removeSigns(response)
    #response = re.sub(r'[^а-яА-Я;.,!?]', '', response) # Clear the response, remains only russian
    response_с = response.split(quote)[-1] #Remove the Quote
    clean_response = remove_duplicates(re.sub(r"\d{4,}", "", response_с)) # Remove the consequent numbers with 4 or more digits
    return clean_response

In [73]:
print("Response:",getResponce())

Response: в бригаде есть несколько батальонов "йерихон". они основном для того чтобы отражать атаки хезов. батальонный уровень это ротные минометы на джипах прицепами (на уровне батальона). если них будет достаточно ракет могут даже накрыть батарею 120мм минометной установки или из состава бригады может быть хуже чем батареи 122 м-109 которые находятся под управлением роты/бат аля рейнджеры... опять все зависит ситуации например после первой ливанской артиллеристы стали очень сильно нервничать когда обстреливали израильские бпла типа 28, как приходилось отвечать свои задачи. теперь вот примеру американцы решили полностью перевести всю бригаду второй эшелон : 1) сократив количество артдивизионов 4х; 3 пехотных батальонах(м113); 5 танковых + отдельный армейский который сможет прикрывать танки непосредственно перед атакой противника.. правда нужно еще иметь возможность поддерживать свой штатную авиацию огнем своего штатного места без необходимости перебрасывать туда часть танков.... короч

Spelling Fix:

In [74]:
from autocorrect import Speller
spell = Speller('ru')
spell_fix_response = spell(getResponce())

In [75]:
print(spell_fix_response)

в бригаде есть несколько батальонов "рихон". они основном для того чтобы отражать атаки уезов. батальонный уровень это ротные минометы на джипах прицепами (на уровне батальона). если них будет достаточно ракет могут даже накрыть батарею 120мм минометной установки или из состава бригады может быть хуже чем батареи 122 м-109 которые находятся под управлением роты/бат аля рейнджеры... опять все зависит ситуации например после первой ливанской артиллеристы стали очень сильно нервничать когда обстреливали израильские была типа 28, как приходилось отвечать свои задачи. теперь вот примеру американцы решили полностью перевести всю бригаду второй эшелон : 1) сократив количество артдивизионов 4х; 3 пехотных батальонах(м113); 5 танковых + отдельный армейский который сможет прикрывать танки непосредственно перед атакой противника.. правда нужно еще иметь возможность поддерживать свой штатную авиацию огнем своего штатного места без необходимости перебрасывать туда часть танков.... короче говоря вся